In [ ]:
import kagglehub
arshkon_linkedin_job_postings_path = kagglehub.dataset_download('arshkon/linkedin-job-postings')

print('Data source import complete.')


100%|██████████| 159M/159M [00:01<00:00, 131MB/s]

Extracting files...


Data source import complete.


하이브리드 기법을 활용해 일자리 추천 시스템을 구축합니다.
텍스트 벡터화, 스킬 임베딩, BERT 기반 의미론적 매칭을 결합하여 지능적인 추천을 진행합니다. 원시 CSV 파일로부터 필터링, 사용자 중심의 로직, 모델 비교까지 포함해 실제적인 추천 시스템을 구현합니다.


1. 라이브러리 불러오기
데이터 처리, 자연어 처리(NLP), 모델링 및 평가를 위한 필수 Python 라이브러리를 불러옵니다.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import shap
import warnings
warnings.filterwarnings('ignore')

SV 파일 불러오기

핵심 데이터셋을 불러옵니다.

job_postings.csv – 직무 제목, 설명, 급여, 경력 수준, 위치 등의 정보를 포함합니다.

job_skills.csv – 각 job_id에 연결된 skill_abr(스킬 약어)를 매핑합니다.

skills.csv – MRKT와 같은 약어를 Marketing과 같은 사람이 읽을 수 있는 스킬 이름으로 변환합니다.

In [ ]:
job_df = pd.read_csv("/content/sample_data/postings.csv")
job_skills = pd.read_csv("/content/sample_data/job_skills.csv")
skills = pd.read_csv("/content/sample_data/skills.csv")

데이터 필터링 및 샘플링

데이터 품질을 향상과 처리 시간을 줄이기 위해 기본적인 필터링 및 샘플링 로직을 적용했습니다.

결측값 제거 – 주요 필드에 null 값이 있는 채용 공고는 제거했습니다.

샘플링 – 실험을 위해 무작위로 1,000개의 채용 공고를 선택했습니다.

In [ ]:
# 주요 컬럼에 결측치(null)가 있는 행 제거
filtered_df = job_df.dropna(subset=['title', 'description', 'location', 'skills_desc', 'formatted_experience_level'])

# 필터링된 데이터에서 무작위로 1,000개 샘플링 (재현 가능하도록 random_state 고정)
sampled_df = filtered_df.sample(n=1000, random_state=42).reset_index(drop=True)

기술 데이터 병합

의미 있는 유사도 계산이 가능하도록 각 채용 공고에 사람이 이해할 수 있는 기술 이름을 추가합니다.

job_skills.csv와 skills.csv를 skill_abr(기술 약어)를 기준으로 조인합니다.

기술 정보를 채용 공고와 결합하여 직무 프로필을 구성합니다.

"Other"와 같이 일반적인 기술만 포함된 채용 공고는 필터링하여 제외합니다.


In [ ]:
# job_skills와 skills를 스킬 약어(skill_abr)를 기준으로 병합
job_skills_merged = job_skills.merge(skills, on='skill_abr', how='left')

# 샘플링된 job_id만 선택
sampled_job_ids = sampled_df['job_id']
sampled_skills = job_skills_merged[job_skills_merged['job_id'].isin(sampled_job_ids)]

# job_id별로 skill_name을 합쳐 하나의 문자열로 변환
job_skill_text = sampled_skills.groupby('job_id')['skill_name'].apply(lambda x: ' '.join(x)).reset_index()
job_skill_text.columns = ['job_id', 'skills_text']  # 컬럼 이름 재설정

# 샘플 데이터프레임에 스킬 텍스트 병합
sampled_df = sampled_df.merge(job_skill_text, on='job_id', how='left')

# 스킬 정보가 없는 경우 빈 문자열로 채움
sampled_df['skills_text'] = sampled_df['skills_text'].fillna('')

# 하이브리드 추천 시스템

**1단계: TF-IDF 기반 유사도 계산**

이 단계에서는 기술(스킬), 직무 제목, 경력 수준을 결합하여 TF-IDF를 활용한 의미 있는 직무 표현을 구성합니다.

이는 하이브리드 추천 시스템의 기초 레이어를 형성합니다.

In [ ]:
# 스킬, 직무 제목, 경력 수준을 결합한 텍스트 생성
sampled_df['hybrid_text'] = (
    sampled_df['skills_text'].fillna('') + ' ' +
    sampled_df['title'].fillna('') + ' ' +
    sampled_df['formatted_experience_level'].fillna('')
)

# TF-IDF 벡터화 (불용어는 영어 기준으로 제거)
hybrid_vectorizer = TfidfVectorizer(stop_words='english')
hybrid_matrix = hybrid_vectorizer.fit_transform(sampled_df['hybrid_text'])  # TF-IDF 행렬 생성

# 코사인 유사도 계산
hybrid_sim_matrix = cosine_similarity(hybrid_matrix, hybrid_matrix)


TF-IDF 엔진 작동 방식

hybrid_text라는 통합 텍스트 필드 생성:

기술 정보 (skills_text),
직무 제목 (title),
경력 수준 (formatted_experience_level)
이 세 가지를 하나로 병합함

TF-IDF 벡터화 적용:
→ 자주 등장하지만 의미 없는 단어는 낮게,
→ 드물지만 정보량이 많은 단어에 가중치를 부여함

각 채용 공고를 **벡터(숫자 행렬)**로 변환
→ 텍스트의 구조와 내용을 수치적으로 표현함

코사인 유사도로 두 채용 공고 간의 유사성 정도를 비교
→ 값이 1에 가까울수록 더 비슷함

**2단계: Count Vectorizer**

이번에는 TF-IDF보다 더 단순한 방식인 Count Vectorizer를 실험해봅니다.

이 방법 역시 기술, 직무 제목, 경력 수준을 결합하지만,
단어의 중요도보다는 **단순한 등장 빈도(횟수)**에 초점을 맞춥니다.

In [ ]:
# Count Vectorizer로 텍스트 벡터화 (단어 출현 빈도 기반)
alt_vectorizer = CountVectorizer()
alt_matrix = alt_vectorizer.fit_transform(sampled_df['hybrid_text'])

# 코사인 유사도로 채용 공고 간 유사도 계산
alt_sim_matrix = cosine_similarity(alt_matrix, alt_matrix)

Recommendation Engine: 핵심 함수 구성

유사도 계산 로직 정의, 직무 순위 지정, 개인화된 추천 결과 반환까지
모든 과정을 재사용 가능한 함수로 구성합니다.

In [ ]:
# 직무 제목 키워드로 추천 실행
def recommend_by_hybrid_title(title_keyword, top_n=5, method='tfidf'):
    # 제목에 키워드가 포함된 채용 공고 찾기
    matches = sampled_df[sampled_df['title'].str.lower().str.contains(title_keyword.lower())]
    if matches.empty:
        print("해당 키워드를 포함한 직무 제목이 없습니다.")
        return

    # 첫 번째 매칭된 공고의 인덱스 선택
    job_index = matches.index[0]
    print(f"매칭된 직무: {sampled_df.iloc[job_index]['title']} (Index: {job_index})\n")

    # 추천 함수 호출 (아래에 정의되어 있어야 함)
    recommend_by_hybrid(job_index, top_n, method)


In [ ]:
# 유사도 기반 직무 추천 함수
def recommend_by_hybrid(job_index, top_n=5, method='tfidf'):
    # TF-IDF 또는 Count Vectorizer 기반 유사도 행렬 선택
    sim_matrix = hybrid_sim_matrix if method == 'tfidf' else alt_sim_matrix

    # 선택한 공고에 대한 유사도 점수 계산
    sim_scores = list(enumerate(sim_matrix[job_index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)  # 유사도 순으로 정렬
    top_jobs = sim_scores[1:top_n+1]  # 자기 자신 제외하고 상위 N개 선택

    print(f"추천 기준 직무: {sampled_df.iloc[job_index]['title']} ({method} 기반)\n")
    for idx, score in top_jobs:
        print(f"제목: {sampled_df.iloc[idx]['title']}")
        print(f"위치: {sampled_df.iloc[idx]['location']}")
        print(f"경력 수준: {sampled_df.iloc[idx]['formatted_experience_level']}")
        print(f"기술: {sampled_df.iloc[idx]['skills_text']}")
        print(f"유사도 점수: {score:.4f}\n")


In [ ]:
# "Data Analyst"라는 키워드를 포함하는 직무에서
# TF-IDF 기반 유사도를 바탕으로 상위 5개의 유사한 직무 추천 실행
recommend_by_hybrid_title("Data Analyst", top_n=5, method='tfidf')

매칭된 직무: Business Data Analyst (Index: 126)

추천 기준 직무: Business Data Analyst (tfidf 기반)

제목: Business Analyst
위치: Boston, MA
경력 수준: Mid-Senior level
기술: Research Analyst Information Technology
유사도 점수: 0.6968

제목: Business Analyst
위치: Atlanta, GA
경력 수준: Mid-Senior level
기술: Research Analyst Information Technology
유사도 점수: 0.6968

제목: Data Governance Analyst
위치: Charlotte, NC
경력 수준: Entry level
기술: Information Technology
유사도 점수: 0.6556

제목: Lead Data Engineer
위치: Nashville, TN
경력 수준: Mid-Senior level
기술: Information Technology
유사도 점수: 0.5943

제목: Data Engineer
위치: Bellevue, WA
경력 수준: Entry level
기술: Information Technology
유사도 점수: 0.5941



In [ ]:
# "Engineer" 키워드를 포함한 직무를 기준으로
# Count Vectorizer 기반 유사도 행렬로 상위 5개 유사 직무 추천
recommend_by_hybrid_title("Engineer", top_n=5, method='count')

# "Marketing" 키워드를 포함한 직무를 기준으로
# TF-IDF 기반 유사도 행렬로 기본 상위 5개 유사 직무 추천
recommend_by_hybrid_title("Marketing", method='tfidf')


매칭된 직무: Senior Network & Security Engineer (Index: 27)

추천 기준 직무: Senior Network & Security Engineer (count 기반)

제목: Senior Network Security Engineer
위치: Charlotte, NC
경력 수준: Mid-Senior level
기술: Information Technology
유사도 점수: 1.0000

제목: Lead Network Engineer
위치: East Chicago, IN
경력 수준: Mid-Senior level
기술: Information Technology
유사도 점수: 0.8528

제목: Senior Highway Engineer
위치: Tampa, FL
경력 수준: Mid-Senior level
기술: Engineering Information Technology
유사도 점수: 0.8182

제목: Senior Engineer Manager
위치: New Orleans, LA
경력 수준: Mid-Senior level
기술: Engineering Information Technology
유사도 점수: 0.8182

제목: Senior Manufacturing Engineer
위치: East Hartford, CT
경력 수준: Mid-Senior level
기술: Engineering Information Technology
유사도 점수: 0.8182

매칭된 직무: Marketing Intern (Index: 11)

추천 기준 직무: Marketing Intern (tfidf 기반)

제목: Marketing Intern
위치: Concord, OH
경력 수준: Internship
기술: Marketing Sales
유사도 점수: 1.0000

제목: Communications and Marketing Manager
위치: Queens, New York, United States
경력 수준: Mid-Senior level

In [ ]:
recommend_by_hybrid_title("Astronaut")

해당 키워드를 포함한 직무 제목이 없습니다.


"Astronaut"는 일반적인 채용 공고 데이터셋에 포함될 가능성이 매우 낮기 때문에 sampled_df 내 title 컬럼에 해당 키워드가 존재하지 않아서 조건문에서 empty로 처리

In [ ]:
# 상위 10개의 직무 제목, 기술 목록, 경력 수준을 확인
sampled_df[['title', 'skills_text', 'formatted_experience_level']].head(10)

,title,skills_text,formatted_experience_level
0,Customer Service Representative,Other,Entry level
1,Registered Nurse - RN - LTAC,Health Care Provider,Mid-Senior level
2,Patient Care Coordinator - Lone Tree Family Pr...,Health Care Provider,Entry level
3,Accounts Payable Specialist,Accounting/Auditing Finance,Entry level
4,Volunteer: Host for International Visitors,Design Art/Creative Information Technology,Associate
5,Senior Project Drafter,Design Art/Creative Information Technology,Mid-Senior level
6,Recreation Specialist- Part Time,Other,Mid-Senior level
7,Grain Merchandiser,Marketing Sales,Entry level
8,Tax Senior Accountant,Accounting/Auditing Finance,Mid-Senior level
9,Registered Nurse - RN - ER (Emergency Room),Health Care Provider,Mid-Senior level


In [ ]:
# 0번 인덱스에 해당하는 채용 공고를 기준으로
# TF-IDF 유사도 기반 상위 5개 직무 추천 실행
recommend_by_hybrid(job_index=0, top_n=5, method='tfidf')

추천 기준 직무: Customer Service Representative (tfidf 기반)

제목: Customer Service/ Data Entry
위치: Fallon, MT
경력 수준: Entry level
기술: Other
유사도 점수: 0.6950

제목: Customer Service Representative - Specialty Paint Division
위치: Illinois, United States
경력 수준: Mid-Senior level
기술: Other
유사도 점수: 0.5566

제목: Logistics Account Executive
위치: Chattanooga, TN
경력 수준: Entry level
기술: Customer Service
유사도 점수: 0.5039

제목: Customer Service Quality Specialist
위치: Litchfield Park, AZ
경력 수준: Associate
기술: Quality Assurance
유사도 점수: 0.4047

제목: Service Coordinator
위치: West Chester, OH
경력 수준: Entry level
기술: Other
유사도 점수: 0.3950



추천 결과 내보내기

개인화된 채용 추천 결과를 .csv 파일로 저장하여
분석, 공유 또는 배포에 활용할 수 있도록 합니다

In [ ]:
# 추천 결과를 CSV 파일로 저장하는 함수
def save_recommendations_to_csv(job_index, top_n=5, method='tfidf', filename='recommended_jobs.csv'):
    # TF-IDF 또는 Count Vectorizer 기반 유사도 행렬 선택
    sim_matrix = hybrid_sim_matrix if method == 'tfidf' else alt_sim_matrix

    # 해당 인덱스 기준 유사도 점수 계산 및 정렬
    sim_scores = list(enumerate(sim_matrix[job_index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    top_jobs = sim_scores[1:top_n+1]  # 자기 자신 제외

    recs = []
    for idx, score in top_jobs:
        recs.append({
            'Title': sampled_df.iloc[idx]['title'],
            'Location': sampled_df.iloc[idx]['location'],
            'Experience': sampled_df.iloc[idx]['formatted_experience_level'],
            'Skills': sampled_df.iloc[idx]['skills_text'],
            'Similarity Score': round(score, 4)
        })

    # 결과를 CSV로 저장
    pd.DataFrame(recs).to_csv(filename, index=False)
    print(f"상위 {top_n}개 추천 결과가 '{filename}' 파일로 저장되었습니다.")


In [ ]:
# save_recommendations_to_csv(0, top_n=5, method='tfidf')

사용자 중심 필터 추가:
추천 결과를 개인의 선호와 필요에 맞게 정교화합니다.

적용 가능한 필터:

위치(Location):
사용자가 선호하는 도시 또는 국가 기준으로 결과 제한

경력 수준(Experience Level):
Entry-level, Mid-level, Senior 등 사용자의 경력에 맞는 직무만 추천

근무 형태(Work Type):
Full-time, Contract, Part-time 등 원하는 고용 형태에 따라 필터링

In [ ]:
# 사용자 선호에 따른 필터를 적용한 직무 추천 함수
def recommend_with_filters(job_index, top_n=5, method='tfidf', location=None, experience=None, work_type=None):
    # TF-IDF 또는 Count 기반 유사도 행렬 선택
    sim_matrix = hybrid_sim_matrix if method == 'tfidf' else alt_sim_matrix

    # 유사도 점수 계산 후 내림차순 정렬
    sim_scores = list(enumerate(sim_matrix[job_index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    filtered_jobs = []
    for idx, score in sim_scores[1:]:  # 자기 자신 제외
        row = sampled_df.iloc[idx]

        # 위치 필터
        if location and location.lower() not in str(row['location']).lower():
            continue
        # 경력 수준 필터
        if experience and experience.lower() not in str(row['formatted_experience_level']).lower():
            continue
        # 근무 형태 필터 (컬럼이 없을 수 있으므로 get 사용)
        if work_type and work_type.lower() not in str(row.get('formatted_work_type', '')).lower():
            continue

        filtered_jobs.append((idx, score))
        if len(filtered_jobs) == top_n:
            break

    if not filtered_jobs:
        print("❌ 필터에 맞는 추천 결과가 없습니다.")
        return

    print(f"추천 기준 직무: {sampled_df.iloc[job_index]['title']} ({method} 기반, 필터 적용됨)\n")
    for idx, score in filtered_jobs:
        print(f"제목: {sampled_df.iloc[idx]['title']}")
        print(f"위치: {sampled_df.iloc[idx]['location']}")
        print(f"경력 수준: {sampled_df.iloc[idx]['formatted_experience_level']}")
        print(f"근무 형태: {sampled_df.iloc[idx].get('formatted_work_type', 'N/A')}")
        print(f"기술: {sampled_df.iloc[idx]['skills_text']}")
        print(f"유사도 점수: {score:.4f}\n")


In [ ]:
recommend_with_filters(
    job_index=1,
    top_n=5,
    method='tfidf',
    location="New York",           # 위치 필터: "New York" 포함
    experience="Entry level",      # 경력 수준 필터: "Entry level" 포함
    work_type="Full-time"          # 근무 형태 필터: "Full-time" 포함
)

추천 기준 직무: Registered Nurse - RN - LTAC (tfidf 기반, 필터 적용됨)

제목: Social Worker
위치: New York, NY
경력 수준: Entry level
근무 형태: Full-time
기술: Other
유사도 점수: 0.0194

제목: Outreach Worker
위치: New York, NY
경력 수준: Entry level
근무 형태: Full-time
기술: Administrative
유사도 점수: 0.0170

제목: Project Accountant
위치: New York, NY
경력 수준: Entry level
근무 형태: Full-time
기술: Accounting/Auditing Finance
유사도 점수: 0.0146

제목: Cook  -  Private School
위치: New York, NY
경력 수준: Entry level
근무 형태: Full-time
기술: Management Manufacturing
유사도 점수: 0.0134

제목: Express Lane Technician
위치: New York, United States
경력 수준: Entry level
근무 형태: Full-time
기술: Engineering Information Technology
유사도 점수: 0.0128



In [ ]:
# 직무 제목 기반 + 사용자 필터를 포함한 추천 실행 함수
def recommend_with_filters_by_title(
    job_title,
    top_n=5,
    method='tfidf',
    location=None,
    experience=None,
    work_type=None
):
    # 제목에 키워드를 포함한 채용 공고 탐색
    matches = sampled_df[sampled_df['title'].str.lower().str.contains(job_title.lower())]

    if matches.empty:
        print("해당 키워드를 포함한 직무 제목이 없습니다.")
        return

    # 첫 번째 매칭된 공고 인덱스 사용
    job_index = matches.index[0]
    print(f"매칭된 직무: {sampled_df.iloc[job_index]['title']} (Index: {job_index})\n")

    # 필터 포함 추천 함수 호출
    recommend_with_filters(
        job_index=job_index,
        top_n=top_n,
        method=method,
        location=location,
        experience=experience,
        work_type=work_type
    )


In [ ]:
recommend_with_filters_by_title(
    job_title="Engineer",              # "Engineer"라는 키워드가 포함된 직무 제목 탐색
    top_n=5,                           # 상위 5개 추천
    method='tfidf',                   # TF-IDF 기반 유사도 계산
    location="New York",              # 위치 필터
    experience="Entry level",         # 경력 수준 필터
    work_type="Full-time"             # 근무 형태 필터
)


매칭된 직무: Senior Network & Security Engineer (Index: 27)

추천 기준 직무: Senior Network & Security Engineer (tfidf 기반, 필터 적용됨)

제목: Express Lane Technician
위치: New York, United States
경력 수준: Entry level
근무 형태: Full-time
기술: Engineering Information Technology
유사도 점수: 0.1151

제목: Social Worker
위치: New York, NY
경력 수준: Entry level
근무 형태: Full-time
기술: Other
유사도 점수: 0.0185

제목: Outreach Worker
위치: New York, NY
경력 수준: Entry level
근무 형태: Full-time
기술: Administrative
유사도 점수: 0.0163

제목: Project Accountant
위치: New York, NY
경력 수준: Entry level
근무 형태: Full-time
기술: Accounting/Auditing Finance
유사도 점수: 0.0140

제목: Cook  -  Private School
위치: New York, NY
경력 수준: Entry level
근무 형태: Full-time
기술: Management Manufacturing
유사도 점수: 0.0127



Semantic Hybrid Recommender with BERT

단어 수를 넘어서 의미 수준의 매칭을 구현하기 위해
**딥러닝 기반 의미 임베딩(BERT)**을 활용한 추천 시스템을 구축합니다.

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# 사전 학습된 Sentence-BERT 모델 불러오기
bert_model = SentenceTransformer('all-MiniLM-L6-v2')

# hybrid_text에 대해 BERT 임베딩 생성
bert_embeddings = bert_model.encode(sampled_df['hybrid_text'].tolist(), show_progress_bar=True)

# 임베딩 간 코사인 유사도 계산
bert_sim_matrix = cosine_similarity(bert_embeddings, bert_embeddings)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

In [ ]:
# BERT 기반 의미 유사도에 따른 직무 추천 함수
def recommend_by_bert(job_index, top_n=5):
    # 해당 인덱스에 대해 BERT 유사도 점수 계산
    sim_scores = list(enumerate(bert_sim_matrix[job_index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)  # 유사도 순 정렬
    top_jobs = sim_scores[1:top_n+1]  # 자기 자신 제외하고 상위 N개 선택

    print(f"추천 기준 직무: {sampled_df.iloc[job_index]['title']} (BERT 기반)\n")
    for idx, score in top_jobs:
        print(f"제목: {sampled_df.iloc[idx]['title']}")
        print(f"위치: {sampled_df.iloc[idx]['location']}")
        print(f"경력 수준: {sampled_df.iloc[idx]['formatted_experience_level']}")
        print(f"기술: {sampled_df.iloc[idx]['skills_text']}")
        print(f"유사도 점수: {score:.4f}\n")


In [ ]:
recommend_by_bert(2)

추천 기준 직무: Patient Care Coordinator - Lone Tree Family Practice (BERT 기반)

제목: LPN-Family
위치: Davenport, FL
경력 수준: Entry level
기술: Health Care Provider
유사도 점수: 0.6571

제목: Physician - Family Practice
위치: San Marcos, CA
경력 수준: Mid-Senior level
기술: Health Care Provider
유사도 점수: 0.6568

제목: Group Treatment Coordinator
위치: Floral City, FL
경력 수준: Entry level
기술: Health Care Provider
유사도 점수: 0.6444

제목: LICENSED PRACTICAL NURSE - PINEHURST HEALTHCARE & REHABILITATION CENTER
위치: North Carolina, United States
경력 수준: Entry level
기술: Health Care Provider
유사도 점수: 0.6054

제목: REGISTERED NURSE - WOODHAVEN
위치: Lumberton, NC
경력 수준: Mid-Senior level
기술: Health Care Provider
유사도 점수: 0.5996



In [ ]:
# 직무 제목 키워드 기반 BERT 추천 함수
def recommend_by_bert_title(title_keyword, top_n=5):
    # 키워드를 포함한 직무 제목 찾기
    matches = sampled_df[sampled_df['title'].str.lower().str.contains(title_keyword.lower())]

    if matches.empty:
        print("해당 키워드를 포함한 직무 제목이 없습니다.")
        return

    # 첫 번째 매칭된 직무 인덱스
    job_index = matches.index[0]
    print(f"매칭된 직무: {sampled_df.iloc[job_index]['title']} (Index: {job_index})\n")

    # BERT 유사도 계산
    sim_scores = list(enumerate(bert_sim_matrix[job_index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    top_jobs = sim_scores[1:top_n+1]  # 자기 자신 제외 상위 추천

    print(f"BERT 기반 추천 결과:\n")
    for idx, score in top_jobs:
        print(f"제목: {sampled_df.iloc[idx]['title']}")
        print(f"위치: {sampled_df.iloc[idx]['location']}")
        print(f"경력 수준: {sampled_df.iloc[idx]['formatted_experience_level']}")
        print(f"기술: {sampled_df.iloc[idx]['skills_text']}")
        print(f"유사도 점수: {score:.4f}\n")


In [ ]:
recommend_by_bert_title("Data Analyst", top_n=5)

매칭된 직무: Business Data Analyst (Index: 126)

BERT 기반 추천 결과:

제목: Business Analyst
위치: Boston, MA
경력 수준: Mid-Senior level
기술: Research Analyst Information Technology
유사도 점수: 0.8373

제목: Business Analyst
위치: Atlanta, GA
경력 수준: Mid-Senior level
기술: Research Analyst Information Technology
유사도 점수: 0.8373

제목: Prospect Research Analyst
위치: Carmel, IN
경력 수준: Mid-Senior level
기술: Research Analyst Information Technology
유사도 점수: 0.7759

제목: Data Collection Researcher/Specialist
위치: Seattle, WA
경력 수준: Mid-Senior level
기술: Information Technology
유사도 점수: 0.7638

제목: Lead Data Engineer
위치: Nashville, TN
경력 수준: Mid-Senior level
기술: Information Technology
유사도 점수: 0.7322



TF-IDF, Count Vectorizer와 같은 고전적인 텍스트 벡터화 기법과
BERT 임베딩 같은 최신 딥러닝 기술을 결합함으로써,
강력한 하이브리드 일자리 추천 시스템을 구축할 수 있음을 보여줍니다.

스킬, 직무명, 경력 수준을 기반으로 사용자와 채용 공고를 정확하고 유연하게 연결할 수 있으며,
모델은 직관적으로 설명 가능하고 다양한 조건에 따라 확장할 수 있습니다.

또한 이 시스템은 모델 미세 조정이나 사용자 피드백을 통해 더욱 정교해질 수 있으며,
향후에는 협업 필터링(collaborative filtering) 등의 기법을 결합하여
더 높은 정확도와 개인화를 제공합니다.